# 🎫 Chapter 6 练习 1: 智能发券优化 - 把钱花在刀刃上

## 补贴的悖论

你是一家外卖平台的运营经理，手握 100 万张优惠券的预算。一个简单的问题：

> **发给谁？**

传统做法：「发给最可能下单的用户」—— 错！

为什么？因为最可能下单的用户**本来就会下单**，给他们发券只是**白送钱**！

正确做法：发给「**因为你发券才会下单**」的用户。这就是 **Uplift 建模**的核心！

---

## 🎯 学习目标

1. 理解用户分群策略（Persuadables, Sure Things, Lost Causes, Sleeping Dogs）
2. 掌握 Uplift 建模在发券场景的应用
3. 学习 ROI 优化决策方法

## 🌟 四类用户的直觉

想象你开了一家餐厅，要决定给谁发优惠券：

| 用户类型 | 不发券 | 发券 | Uplift | 策略 |
|----------|--------|------|--------|------|
| **Persuadables** (可说服者) | ❌ 不来 | ✅ 来了 | 高正 | 🎯 **重点发券** |
| **Sure Things** (必来者) | ✅ 来了 | ✅ 来了 | 0 | 💰 **浪费钱** |
| **Lost Causes** (流失者) | ❌ 不来 | ❌ 不来 | 0 | 🚫 **别浪费** |
| **Sleeping Dogs** (睡狗) | ✅ 来了 | ❌ 不来 | 负 | ⚠️ **千万别发** |

### Sleeping Dogs 是什么鬼？

有些用户收到促销信息反而会**产生反感**：
- 「这餐厅居然这么便宜，质量是不是有问题？」
- 「发这么多广告，太烦人了，取关！」
- 「居然用优惠券来「骗」我，以后不去了」

这类用户就像「睡着的狗」，千万别去打扰它！

## 📐 核心公式

### Uplift 定义

$$\text{Uplift}(x) = P(Y=1|T=1, X=x) - P(Y=1|T=0, X=x)$$

即：发券时的转化率 - 不发券时的转化率

### ROI 计算

$$\text{ROI} = \frac{\text{增量收入} - \text{成本}}{\text{成本}}$$

$$\text{增量收入} = \text{增量转化数} \times \text{每次转化收入}$$

$$\text{成本} = \text{发券数} \times \text{每张券成本}$$

In [ ]:
# 环境设置
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Tuple, Dict
from sklearn.ensemble import GradientBoostingClassifier

# 设置随机种子
np.random.seed(42)

# 绘图设置
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['font.size'] = 12
plt.rcParams['axes.unicode_minus'] = False
plt.style.use('seaborn-v0_8-whitegrid')

print("环境配置完成! 🎫")

## 练习 1.1: 生成营销场景数据

让我们模拟一个外卖平台的发券实验！

In [ ]:
def generate_simple_marketing_data(
    n_samples: int = 2000,
    seed: int = 42
) -> pd.DataFrame:
    """
    生成简化的营销场景数据
    
    场景: 外卖平台发券实验
    - 特征: 用户年龄、历史订单数、最近活跃度
    - 处理: 是否发券 (T=1 发券, T=0 不发券)
    - 结果: 是否下单 (conversion=1 下单, conversion=0 未下单)
    
    用户类型:
    - Persuadables: 年轻 + 低频用户（被优惠券吸引）
    - Sure Things: 高频用户（本来就爱点外卖）
    - Sleeping Dogs: 低频老用户（反感促销）
    - Lost Causes: 其他
    """
    np.random.seed(seed)
    
    # TODO: 生成用户特征
    # age: 18-65 岁均匀分布
    age = None  # 你的代码
    
    # order_freq: 历史订单数，泊松分布 lambda=5
    order_freq = None  # 你的代码
    
    # days_since_last: 距上次下单天数，指数分布 scale=10
    days_since_last = None  # 你的代码
    
    # TODO: 随机分配处理 (50% 概率发券)
    T = None  # 你的代码
    
    # 确定用户类型
    user_type = []
    conversion = []
    
    for i in range(n_samples):
        # TODO: 根据规则判断用户类型
        # Persuadables: 年轻 (age < 30) + 低频 (order_freq < 5)
        # Sure Things: 高频 (order_freq >= 8)
        # Sleeping Dogs: 非常低频 (order_freq < 2) + 不年轻 (age >= 40)
        # Lost Causes: 其他
        
        if age is not None:
            if age[i] < 30 and order_freq[i] < 5:
                user_type.append('Persuadables')
            elif order_freq[i] >= 8:
                user_type.append('Sure Things')
            elif order_freq[i] < 2 and age[i] >= 40:
                user_type.append('Sleeping Dogs')
            else:
                user_type.append('Lost Causes')
        else:
            user_type.append('Unknown')
        
        # TODO: 生成转化结果
        # 基线转化率: 0.1 + 0.02 * min(order_freq, 10)
        if age is not None and order_freq is not None and T is not None:
            base_prob = 0.1 + 0.02 * min(order_freq[i], 10)
            
            # 根据用户类型调整转化率
            if user_type[i] == 'Persuadables':
                # 发券效果很好：+0.25
                prob = base_prob + (0.25 if T[i] == 1 else 0)
            elif user_type[i] == 'Sure Things':
                # 本来就会买：基线高，发券效果小
                prob = base_prob + 0.3 + (0.03 if T[i] == 1 else 0)
            elif user_type[i] == 'Sleeping Dogs':
                # 发券反而降低：-0.1
                prob = base_prob + 0.15 + (-0.1 if T[i] == 1 else 0)
            else:  # Lost Causes
                # 发不发都不买
                prob = base_prob + (0.01 if T[i] == 1 else 0)
            
            # 确保概率在 [0, 1] 范围内
            prob = np.clip(prob, 0, 1)
            conversion.append(np.random.binomial(1, prob))
        else:
            conversion.append(0)
    
    return pd.DataFrame({
        'age': age,
        'order_freq': order_freq,
        'days_since_last': days_since_last,
        'T': T,
        'conversion': conversion,
        'user_type': user_type
    })

In [ ]:
# 测试数据生成
df = generate_simple_marketing_data(n_samples=5000)

if df is not None and df['age'].iloc[0] is not None:
    print("营销数据生成成功! 🎉")
    print(f"\n样本量: {len(df)}")
    print(f"发券率: {df['T'].mean():.2%}")
    print(f"整体转化率: {df['conversion'].mean():.2%}")
    
    print(f"\n=== 用户类型分布 ===")
    print(df['user_type'].value_counts())
    
    # 计算各类型的真实 Uplift
    print(f"\n=== 各用户类型的真实 Uplift ===")
    for user_type in df['user_type'].unique():
        mask = df['user_type'] == user_type
        treat_conv = df.loc[mask & (df['T']==1), 'conversion'].mean()
        control_conv = df.loc[mask & (df['T']==0), 'conversion'].mean()
        uplift = treat_conv - control_conv
        print(f"  {user_type}: {uplift:+.3f}")
else:
    print("[TODO] 请完成 generate_simple_marketing_data 函数")

In [ ]:
# 可视化用户类型
if df is not None and df['age'].iloc[0] is not None:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    colors = {
        'Persuadables': '#2ecc71',
        'Sure Things': '#3498db',
        'Lost Causes': '#95a5a6',
        'Sleeping Dogs': '#e74c3c'
    }
    
    # 用户分布
    for user_type in df['user_type'].unique():
        mask = df['user_type'] == user_type
        axes[0].scatter(
            df.loc[mask, 'age'],
            df.loc[mask, 'order_freq'],
            alpha=0.5,
            c=colors.get(user_type, 'gray'),
            label=user_type,
            s=30
        )
    axes[0].set_xlabel('年龄')
    axes[0].set_ylabel('历史订单数')
    axes[0].set_title('用户类型在特征空间的分布')
    axes[0].legend()
    
    # Uplift 对比
    uplift_data = []
    for user_type in ['Persuadables', 'Sure Things', 'Lost Causes', 'Sleeping Dogs']:
        mask = df['user_type'] == user_type
        if mask.sum() > 0:
            treat_conv = df.loc[mask & (df['T']==1), 'conversion'].mean()
            control_conv = df.loc[mask & (df['T']==0), 'conversion'].mean()
            uplift_data.append({
                'type': user_type,
                'uplift': treat_conv - control_conv,
                'treat_conv': treat_conv,
                'control_conv': control_conv
            })
    
    if len(uplift_data) > 0:
        uplift_df = pd.DataFrame(uplift_data)
        x = range(len(uplift_df))
        
        bars = axes[1].bar(x, uplift_df['uplift'], 
                          color=[colors.get(t, 'gray') for t in uplift_df['type']])
        axes[1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
        axes[1].set_xticks(x)
        axes[1].set_xticklabels(uplift_df['type'], rotation=15)
        axes[1].set_ylabel('Uplift (转化率提升)')
        axes[1].set_title('各用户类型的真实 Uplift')
        
        # 添加数值标签
        for bar, val in zip(bars, uplift_df['uplift']):
            axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height(),
                        f'{val:+.3f}', ha='center', va='bottom' if val > 0 else 'top')
    
    plt.tight_layout()
    plt.show()

## 练习 1.2: 训练 Uplift 模型

使用 T-Learner (Two-Model) 方法训练 Uplift 模型。

In [ ]:
class SimpleUpliftModel:
    """
    简单的 Uplift 模型 (T-Learner 方法)
    
    思想: 分别训练控制组和处理组的模型，差值即为 Uplift
    """
    
    def __init__(self):
        self.model_control = GradientBoostingClassifier(
            n_estimators=50, max_depth=4, random_state=42
        )
        self.model_treatment = GradientBoostingClassifier(
            n_estimators=50, max_depth=4, random_state=43
        )
    
    def fit(self, X: np.ndarray, T: np.ndarray, Y: np.ndarray):
        """
        训练 Uplift 模型
        """
        # TODO: 分离控制组和处理组数据
        mask_control = None  # 你的代码: T == 0
        mask_treatment = None  # 你的代码: T == 1
        
        # TODO: 训练两个模型
        # self.model_control.fit(X[mask_control], Y[mask_control])
        # self.model_treatment.fit(X[mask_treatment], Y[mask_treatment])
        
        # 你的代码
        
        return self
    
    def predict_uplift(self, X: np.ndarray) -> np.ndarray:
        """
        预测 Uplift
        """
        # TODO: 分别预测两组的转化概率
        prob_treatment = None  # 你的代码
        prob_control = None    # 你的代码
        
        # TODO: Uplift = 处理组概率 - 控制组概率
        uplift = None  # 你的代码
        
        return uplift

In [ ]:
# 训练 Uplift 模型
if df is not None and df['age'].iloc[0] is not None:
    X = df[['age', 'order_freq', 'days_since_last']].values
    T = df['T'].values
    Y = df['conversion'].values
    
    try:
        model = SimpleUpliftModel()
        model.fit(X, T, Y)
        uplift_pred = model.predict_uplift(X)
        
        if uplift_pred is not None:
            print("Uplift 模型训练成功! 🎉")
            print(f"\nUplift 预测统计:")
            print(f"  范围: [{uplift_pred.min():.3f}, {uplift_pred.max():.3f}]")
            print(f"  均值: {uplift_pred.mean():.3f}")
            print(f"  标准差: {uplift_pred.std():.3f}")
            
            # 与真实用户类型对比
            df['uplift_pred'] = uplift_pred
            print(f"\n各用户类型的预测 Uplift 均值:")
            for user_type in ['Persuadables', 'Sure Things', 'Lost Causes', 'Sleeping Dogs']:
                mean_uplift = df.loc[df['user_type']==user_type, 'uplift_pred'].mean()
                print(f"  {user_type}: {mean_uplift:+.3f}")
        else:
            print("[TODO] 请完成 predict_uplift 方法")
    except Exception as e:
        print(f"[TODO] 模型训练失败: {e}")

## 练习 1.3: 用户分群

根据 Uplift 预测分数将用户分群。

In [ ]:
def segment_users(
    df: pd.DataFrame,
    uplift_scores: np.ndarray,
    high_threshold: float = 0.1,
    low_threshold: float = 0.02
) -> pd.DataFrame:
    """
    根据 Uplift 分数将用户分群
    
    分群规则:
    - High Uplift: uplift >= high_threshold (建议发券)
    - Low Uplift: 0 < uplift < low_threshold (可发可不发)
    - Negative Uplift: uplift <= 0 (千万别发!)
    """
    df = df.copy()
    df['uplift_score'] = uplift_scores
    
    # TODO: 实现分群规则
    segments = []
    for i in range(len(df)):
        uplift = uplift_scores[i]
        
        # 你的代码: 根据 uplift 判断分群
        if uplift >= high_threshold:
            segment = 'High Uplift (Target!)'
        elif uplift >= low_threshold:
            segment = 'Medium Uplift'
        elif uplift > 0:
            segment = 'Low Uplift'
        else:
            segment = 'Negative Uplift (Avoid!)'
        
        segments.append(segment)
    
    df['segment'] = segments
    return df

In [ ]:
# 用户分群
if 'uplift_pred' in dir() and uplift_pred is not None:
    df_segmented = segment_users(df, uplift_pred)
    
    print("用户分群结果:")
    print("="*50)
    print(f"\n分群分布:")
    print(df_segmented['segment'].value_counts())
    
    print(f"\n各群组统计:")
    for segment in df_segmented['segment'].unique():
        mask = df_segmented['segment'] == segment
        avg_uplift = df_segmented.loc[mask, 'uplift_score'].mean()
        
        # 计算真实转化率
        treat_conv = df_segmented.loc[mask & (df_segmented['T']==1), 'conversion'].mean()
        control_conv = df_segmented.loc[mask & (df_segmented['T']==0), 'conversion'].mean()
        real_uplift = treat_conv - control_conv
        
        print(f"\n  {segment}:")
        print(f"    样本数: {mask.sum()}")
        print(f"    预测 Uplift: {avg_uplift:+.3f}")
        print(f"    真实 Uplift: {real_uplift:+.3f}")

## 练习 1.4: ROI 计算

计算不同发券策略的投资回报率。

In [ ]:
def calculate_roi_simple(
    df: pd.DataFrame,
    treatment_mask: np.ndarray,
    revenue_per_conversion: float = 100,
    cost_per_coupon: float = 15
) -> Dict[str, float]:
    """
    计算发券策略的 ROI
    
    Args:
        df: 数据
        treatment_mask: 哪些用户应该发券
        revenue_per_conversion: 每次转化的收入 (元)
        cost_per_coupon: 每张券的成本 (元)
    """
    # 发券的用户
    treated_users = df[treatment_mask == 1]
    
    # 在这些用户中，计算实际 uplift
    # (这里用随机分配的数据来估计)
    treat_conv = treated_users.loc[treated_users['T']==1, 'conversion'].mean()
    control_conv = treated_users.loc[treated_users['T']==0, 'conversion'].mean()
    
    if np.isnan(treat_conv) or np.isnan(control_conv):
        actual_uplift = 0
    else:
        actual_uplift = treat_conv - control_conv
    
    # TODO: 计算 ROI
    n_coupons = treatment_mask.sum()  # 发券数量
    expected_conversions = None  # 预期增量转化数: n_coupons * actual_uplift
    revenue = None  # 收入: expected_conversions * revenue_per_conversion
    cost = None     # 成本: n_coupons * cost_per_coupon
    roi = None      # ROI: (revenue - cost) / cost
    
    return {
        'roi': roi,
        'revenue': revenue,
        'cost': cost,
        'n_coupons': n_coupons,
        'expected_conversions': expected_conversions,
        'actual_uplift': actual_uplift
    }

In [ ]:
# 测试 ROI 计算
if 'uplift_pred' in dir() and uplift_pred is not None:
    # 策略: 给 Uplift > 0.05 的用户发券
    test_mask = (uplift_pred > 0.05).astype(int)
    
    roi_result = calculate_roi_simple(df, test_mask)
    
    if roi_result['roi'] is not None:
        print("ROI 计算结果:")
        print("="*40)
        print(f"  发券人数: {roi_result['n_coupons']}")
        print(f"  实际 Uplift: {roi_result['actual_uplift']:.3f}")
        print(f"  预期增量转化: {roi_result['expected_conversions']:.1f}")
        print(f"  成本: ¥{roi_result['cost']:.0f}")
        print(f"  收入: ¥{roi_result['revenue']:.0f}")
        print(f"  ROI: {roi_result['roi']:.2f}")
    else:
        print("[TODO] 请完成 calculate_roi_simple 函数")

## 练习 1.5: 策略对比

对比三种发券策略：
1. **Random**: 随机发券
2. **High Frequency**: 给高频用户发券（传统做法）
3. **Uplift Model**: 根据 Uplift 分数发券（因果推断）

In [ ]:
def compare_strategies(
    df: pd.DataFrame,
    uplift_scores: np.ndarray,
    budget_fraction: float = 0.3
) -> pd.DataFrame:
    """
    对比不同发券策略的效果
    """
    n_target = int(budget_fraction * len(df))
    results = []
    
    # TODO: 策略 1 - Random
    np.random.seed(42)
    random_idx = np.random.choice(len(df), n_target, replace=False)
    random_mask = np.zeros(len(df))
    random_mask[random_idx] = 1
    random_roi = calculate_roi_simple(df, random_mask)
    if random_roi['roi'] is not None:
        results.append({'strategy': 'Random', **random_roi})
    
    # TODO: 策略 2 - High Frequency
    # 选择 order_freq 最高的用户
    high_freq_idx = np.argsort(df['order_freq'].values)[-n_target:]
    high_freq_mask = np.zeros(len(df))
    high_freq_mask[high_freq_idx] = 1
    high_freq_roi = calculate_roi_simple(df, high_freq_mask)
    if high_freq_roi['roi'] is not None:
        results.append({'strategy': 'High Frequency', **high_freq_roi})
    
    # TODO: 策略 3 - Uplift Model
    # 选择 uplift_scores 最高的用户
    uplift_idx = np.argsort(uplift_scores)[-n_target:]
    uplift_mask = np.zeros(len(df))
    uplift_mask[uplift_idx] = 1
    uplift_roi = calculate_roi_simple(df, uplift_mask)
    if uplift_roi['roi'] is not None:
        results.append({'strategy': 'Uplift Model', **uplift_roi})
    
    return pd.DataFrame(results)

In [ ]:
# 策略对比
if 'uplift_pred' in dir() and uplift_pred is not None:
    print("=" * 60)
    print("策略对比实验")
    print("=" * 60)
    print(f"\n预算: 给 30% 的用户发券")
    
    comparison = compare_strategies(df, uplift_pred, budget_fraction=0.3)
    
    if comparison is not None and len(comparison) > 0:
        print(f"\nROI 对比:")
        display(comparison[['strategy', 'n_coupons', 'actual_uplift', 
                           'revenue', 'cost', 'roi']].round(2))
        
        # 找最佳策略
        best_idx = comparison['roi'].idxmax()
        best_strategy = comparison.loc[best_idx, 'strategy']
        best_roi = comparison.loc[best_idx, 'roi']
        
        print(f"\n最佳策略: {best_strategy}")
        print(f"最佳 ROI: {best_roi:.2f}")
        
        # 可视化
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        colors = ['#95a5a6', '#3498db', '#2ecc71']
        
        # ROI 对比
        bars = axes[0].bar(comparison['strategy'], comparison['roi'], color=colors)
        axes[0].axhline(y=0, color='red', linestyle='--')
        axes[0].set_ylabel('ROI')
        axes[0].set_title('不同策略的 ROI 对比')
        for bar, val in zip(bars, comparison['roi']):
            axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height(),
                        f'{val:.2f}', ha='center', va='bottom')
        
        # Uplift 对比
        bars = axes[1].bar(comparison['strategy'], comparison['actual_uplift'], color=colors)
        axes[1].axhline(y=0, color='red', linestyle='--')
        axes[1].set_ylabel('实际 Uplift')
        axes[1].set_title('不同策略的实际 Uplift 对比')
        for bar, val in zip(bars, comparison['actual_uplift']):
            axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height(),
                        f'{val:.3f}', ha='center', va='bottom')
        
        plt.tight_layout()
        plt.show()
        
        print("\n关键洞察:")
        print("- Random: 随机发券，ROI 一般")
        print("- High Frequency: 给高频用户发券，可能ROI更低（浪费在Sure Things上）")
        print("- Uplift Model: 给可说服用户发券，ROI 最高！")

## 🤔 思考题

In [ ]:
# 思考题 1: 为什么 "Sure Things" 用户会造成补贴浪费?

answer_1 = """
你的答案:



"""
print(answer_1)

In [ ]:
# 思考题 2: 为什么 "Sleeping Dogs" 用户发券反而转化率下降？举一个实际例子。

answer_2 = """
你的答案:



"""
print(answer_2)

In [ ]:
# 思考题 3: 在真实业务中，如何验证 Uplift 模型的准确性?

answer_3 = """
你的答案:



"""
print(answer_3)

In [ ]:
# 思考题 4: 如果预算有限 (比如只能发券给 20% 的用户)，应该如何调整发券策略?

answer_4 = """
你的答案:



"""
print(answer_4)

In [ ]:
# 思考题 5: Uplift 建模相比传统的 "响应率建模" 有什么本质区别?

answer_5 = """
你的答案:



"""
print(answer_5)

## 📊 总结

### 用户分群策略

| 用户类型 | Uplift | 策略 | 业务价值 |
|----------|--------|------|----------|
| Persuadables | 高正 | 重点发券 | 💰 高 ROI |
| Sure Things | ~0 | 不发券 | 💸 节省成本 |
| Lost Causes | ~0 | 不发券 | 💸 节省成本 |
| Sleeping Dogs | 负 | 绝不发券 | ⚠️ 避免损失 |

### Uplift 建模 vs 响应率建模

| 方面 | 响应率建模 | Uplift 建模 |
|------|------------|-------------|
| 目标 | 预测谁会转化 | 预测谁会被影响 |
| 结果 | P(Y=1\|X) | P(Y=1\|T=1,X) - P(Y=1\|T=0,X) |
| 问题 | 找到 Sure Things | 找到 Persuadables |

### ROI 优化公式

$$\text{ROI} = \frac{N \times \text{Uplift} \times \text{Revenue} - N \times \text{Cost}}{N \times \text{Cost}}$$

In [ ]:
print("🎫 恭喜完成智能发券优化练习!")
print("\n你已经学会了:")
print("  ✓ 四类用户的概念和识别")
print("  ✓ T-Learner Uplift 模型")
print("  ✓ 用户分群策略")
print("  ✓ ROI 计算和策略对比")
print("\n下一步: A/B 测试增强 - CUPED!")